In [1]:
import requests
from db import BotDB
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import sqlite3
import random
import re
import time

In [2]:
class Crawler:
    def __init__(self, url):
        self.url = url
    
    def get_soup(self):
#         soup = BeautifulSoup(requests.get(self.url).text, "html.parser")
        driver = webdriver.Chrome()
        driver.get(self.url)
        source_data = driver.page_source
        soup = bs(source_data)
        
        return soup

In [3]:
class Ctrl:
    def __init__(self):
        pass
    
    #n - количество коленьев, которое задает пользователь
    def get_link_and_parse(self, name, n):
        
        #получение ссылки на актера
        url = Parser().get_link_actor(name)
        #получение ссылок на фильмы, в которых он снимался
        link_films = Parser().get_info_actor(url)
        links_emp = []
        
        for link in link_films:
            #получение ссылок на друзей актера
            link_employees = Parser().get_info_film(link)
            for link_emp in link_employees:
                Parser().get_info_actor(link_emp)
                links_emp.append(link_emp)
            print(links_emp)

        for _ in range(n-1):
            links_emp = []
            for link_emp in links_emp:
                #получение ссылок на фильмы, в которых он снимался
                link_films = Parser().get_info_actor(link_emp)

                #обход этих ссылок по одной
                for link in link_films:
                    #получение ссылок на друзей актера
                    link_employees = Parser().get_info_film(link)
                    for link_emp in link_employees:
                        Parser().get_info_actor(link_emp)
                        links_emp.append(link_emp)
                    print(links_emp)

In [4]:
class Parser:  
    def __init__(self):
        self.db = BotDB()
    
    #получение ссылки для определенного актера, имя которого вводит пользователь
    def get_link_actor(self, name):
        
        driver = webdriver.Chrome()
        driver.get('https://ru.kinorium.com/')
        input_tab = driver.find_element(By.CLASS_NAME, 'ui-autocomplete-input')
        input_tab.send_keys(name)
        input_tab.send_keys(Keys.ENTER)
        tab_actor = driver.find_element(By.XPATH, '/html/body/div[6]/div/div[6]/div[1]/div/div/div[2]/div/div[3]/div[1]/h3/a')
        tab_actor.send_keys(Keys.ENTER)
        
        return driver.current_url
    
    #получение информации об актерах, возвращает все ссылки фильмов, в которых снимался данный актер
    def get_info_actor(self, url):
        
        soup = Crawler(url).get_soup()
        time.sleep(7)
        
        id_actor = re.findall(r'\d+', url)
        name = soup.find('div', class_ = 'person-page__title-elements-wrap').text.replace(u'\xa0', u' ').strip()
        
        try:
            birthdate = soup.find('div', class_ = 'person_info').find('meta', itemprop = 'birthDate').get('content')
            
        except Exception as E:
            birthdate = 'no birthdate'
    
        try:
            gender = soup.find('div', class_ = 'person_info').find('meta', itemprop = 'gender').get('content')
            
        except Exception as E:
            gender = 'no gender'

        link_films = []
        id_films = []
        for link in soup.findAll('div', class_ = 'item headlines_type-actor'):
            link_film = 'https://ru.kinorium.com' + link.find('a', class_ = 'filmList__item-title').get('href')
            link_films.append(link_film)
            id_films.append(re.findall(r'\d+',link_film))
            
        #этот массив добавлять в таблицу actor
        info_actor = [id_actor, name, gender, birthdate, id_films]
        if not self.db.actor_exist(id_actor[0]):
            if birthdate == 'no birthdate':
                birthdate=None
            else:
                if re.match(r'\d{4}-0{2}-00', birthdate):
                    birthdate = birthdate.replace('-00', '-01-01')
            info_actor_db = id_actor, name, gender, birthdate, id_films
            self.db.add_actor(info_actor_db)
        print(info_actor)
        
        return link_films
    
    #получение информации о фильме, возвращает ссылки на всех людей, которые учавствовали в съёмках
    def get_info_film(self, link):

        id_film = re.findall(r'\d+', link)
        
        soup = Crawler(link).get_soup()

        name = soup.find('h1', class_ = 'film-page__title-text').text
        time.sleep(7)

        try:
            
            countries = []
            for country in soup.findAll('a', class_ = 'film-page__country-link'):
                countries.append(country.text)

            genres = []
            for genre in soup.findAll('li', itemprop = 'genre'):
                genres.append(genre.text)

            warning = {}

            for par_con in soup.find('tr', class_ = 'film-page__parentalguide-toggle-line').findAll('li'):
                parental_control_count = par_con.find('p').text
                parental_control = par_con.text
                parental_control = parental_control.replace(parental_control_count, '')

                warning[parental_control] = parental_control_count

        except Exception as E:
            warning = 'no parental control'
        
        #этот массив добавлять в таблицу films
        info_films = [name, link, warning, countries, genres]
        
        
        film_id = re.findall(r'\d+',link)
        info_films_db = name, film_id[0], warning, countries, genres
        if not self.db.film_exist(film_id):
            self.db.add_film(info_films_db)
        print(info_films)
        time.sleep(7)
    
        soup_cast = Crawler(link + 'cast').get_soup()
        
        employees = {}
        emp_links = []
        for t in soup_cast.findAll('div', class_='ref-list clearfix'):

            for pos in t.findAll('h1', class_='cast-page__title'):
                position = pos.text.strip()
                employees[position] = []

            for emp in t.findAll('a', class_='cast-page__link-name link-info-persona-type-persona'):
                employee = emp.get('href')
                emp_link = 'https://ru.kinorium.com/' + employee
                emp_links.append(emp_link)
                employee_id = re.findall(r'\d+', employee) 
                employees[position].append(employee_id)


        return emp_links 

In [5]:
Ctrl().get_link_and_parse('Роберт Паттинсон', 2)

[['1030531'], 'Роберт Паттинсон', 'Мужчина', '1986-05-13', [['1660332'], ['1590285'], ['697345'], ['569768'], ['539518'], ['402151'], ['428490'], ['421739'], ['401674'], ['382907'], ['301222'], ['195539']]]
['Король Англии', 'https://ru.kinorium.com/1660332/', {'жестокость/кровь ': 'много', 'пугающие сцены ': 'средне', 'ненормативная лексика ': 'средне', 'алкоголь, наркотики, курение ': 'мало', 'секс/нагота ': 'мало'}, ['Великобритания', 'Венгрия', 'США', 'Австралия'], ['Драма', 'История', 'Биография', 'Военный']]
[['1513313'], 'Дэвид Мишо', 'Мужчина', '1972-11-30', [['1684427'], ['674237'], ['459543'], ['2112348'], ['448660'], ['461699'], ['417795'], ['417947'], ['2598939'], ['385995'], ['2384937'], ['412328'], ['2056910']]]
[['1870994'], 'Тимоти Шаламе', 'Мужчина', '1995-12-27', [['2599724'], ['1996854'], ['1974039'], ['1560686'], ['549200'], ['1554498'], ['723199'], ['384511'], ['722399'], ['706507'], ['642576'], ['561170'], ['460680'], ['466543'], ['425604'], ['458517'], ['90445']]

[['4694203'], 'Стивен Фьюэлл', 'Мужчина', 'no birthdate', [['1660332'], ['1999105'], ['1605560'], ['1482516'], ['2514467']]]
[['3135116'], 'Даниил Эльдаров', 'Мужчина', '1981-10-23', [['10415827'], ['10236820'], ['9943707'], ['9532814'], ['9943742'], ['9597301'], ['2030515'], ['2411501'], ['2328512'], ['2412902'], ['1659099'], ['2023509'], ['2201117'], ['1582013'], ['1693938'], ['2209923'], ['1653066'], ['1622956'], ['1659004'], ['1607128'], ['2736000'], ['2209922'], ['1622980'], ['1580748'], ['1480741'], ['731305'], ['729407'], ['732518'], ['10413862'], ['733330'], ['730096'], ['465522'], ['735094'], ['735011'], ['735802'], ['336182'], ['301945'], ['735611'], ['771214']]]
[['333338'], 'Айван Кэй', 'Мужчина', '1961-07-01', [['2803745'], ['2835064'], ['1602337'], ['2567187'], ['2023913'], ['1624858'], ['2389231'], ['1660332'], ['1629522'], ['643194'], ['1661957'], ['1541757'], ['2062456'], ['629900'], ['703460'], ['647748'], ['502703'], ['424938'], ['590414'], ['548122'], ['517895'], ['

[['4189186'], 'Джек Бандейра', 'Мужчина', 'no birthdate', [['2608271'], ['2380395'], ['2569700'], ['2041895'], ['2023913'], ['2308888'], ['2093365'], ['1700329'], ['1675385'], ['1614766'], ['1677232'], ['1660332'], ['1672104'], ['1629943'], ['1636553'], ['754331'], ['2709294'], ['610681'], ['540691'], ['152463'], ['104732']]]
[['694069'], 'Криштоф Виддер', 'Мужчина', '1988-04-20', [['2750986'], ['741927'], ['1614766'], ['1660332'], ['2009677'], ['1611896'], ['1643188'], ['1693597'], ['1605876'], ['1482662'], ['496469'], ['2461504'], ['2113094'], ['102490'], ['161584']]]
[['368819'], 'Винсент Латорре', 'Мужчина', 'no birthdate', [['1660332'], ['1703044'], ['1448955'], ['1666798'], ['1666383'], ['1645005'], ['1642662'], ['1545395'], ['2594288'], ['2604316'], ['2413261'], ['383404'], ['311320'], ['646252'], ['104732'], ['416861'], ['142663']]]
[['5137127'], 'Bence Bakti', 'Мужчина', 'no birthdate', []]
[['4713580'], 'Бардо Феньвеши', 'Мужчина', 'no birthdate', [['1468380']]]
[['5137128'],

TimeoutException: Message: timeout: Timed out receiving message from renderer: 600.000
  (Session info: chrome=109.0.5414.149)
Stacktrace:
Backtrace:
	(No symbol) [0x01416643]
	(No symbol) [0x013ABE21]
	(No symbol) [0x012ADA9D]
	(No symbol) [0x0129F55A]
	(No symbol) [0x0129F2D8]
	(No symbol) [0x0129DC68]
	(No symbol) [0x0129E512]
	(No symbol) [0x0129D205]
	(No symbol) [0x012A91BC]
	(No symbol) [0x0129D0D5]
	(No symbol) [0x0129EE5A]
	(No symbol) [0x0129DC68]
	(No symbol) [0x0129E512]
	(No symbol) [0x0129D205]
	(No symbol) [0x012A671C]
	(No symbol) [0x0129D0D5]
	(No symbol) [0x0129EE5A]
	(No symbol) [0x0129DC68]
	(No symbol) [0x0129E512]
	(No symbol) [0x0129D205]
	(No symbol) [0x012A40E2]
	(No symbol) [0x0129D0D5]
	(No symbol) [0x0129EE5A]
	(No symbol) [0x0129DC68]
	(No symbol) [0x0129E512]
	(No symbol) [0x0129D205]
	(No symbol) [0x01295606]
	(No symbol) [0x0129D0D5]
	(No symbol) [0x0129C670]
	(No symbol) [0x0129C7BF]
	(No symbol) [0x0129CC94]
	(No symbol) [0x0129CAD8]
	(No symbol) [0x01